In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *
import tensorflow as tf


In [2]:

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values


In [3]:

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')


In [4]:

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=5,
    write_graph=True
)


In [5]:

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)


Epoch 1/50
32/32 - 0s - loss: 0.0341
Epoch 2/50
32/32 - 0s - loss: 0.0047
Epoch 3/50
32/32 - 0s - loss: 0.0018
Epoch 4/50
32/32 - 0s - loss: 9.2330e-04
Epoch 5/50
32/32 - 0s - loss: 6.1460e-04
Epoch 6/50
32/32 - 0s - loss: 3.7571e-04
Epoch 7/50
32/32 - 0s - loss: 2.1783e-04
Epoch 8/50
32/32 - 0s - loss: 1.6105e-04
Epoch 9/50
32/32 - 0s - loss: 1.1720e-04
Epoch 10/50
32/32 - 0s - loss: 9.5048e-05
Epoch 11/50
32/32 - 0s - loss: 9.5865e-05
Epoch 12/50
32/32 - 0s - loss: 9.0560e-05
Epoch 13/50
32/32 - 0s - loss: 8.0128e-05
Epoch 14/50
32/32 - 0s - loss: 5.9059e-05
Epoch 15/50
32/32 - 0s - loss: 5.1484e-05
Epoch 16/50
32/32 - 0s - loss: 4.4296e-05
Epoch 17/50
32/32 - 0s - loss: 3.8885e-05
Epoch 18/50
32/32 - 0s - loss: 3.5744e-05
Epoch 19/50
32/32 - 0s - loss: 3.7022e-05
Epoch 20/50
32/32 - 0s - loss: 4.0824e-05
Epoch 21/50
32/32 - 0s - loss: 3.2192e-05
Epoch 22/50
32/32 - 0s - loss: 3.4584e-05
Epoch 23/50
32/32 - 0s - loss: 2.7905e-05
Epoch 24/50
32/32 - 0s - loss: 2.6511e-05
Epoch 25/50
3

In [6]:

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_testing_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))



The mean squared error (MSE) for the test data set is: 0.00014193484093993902


### Exporting model

In [10]:

model_builder = tf.saved_model.builder.SavedModelBuilder("exported_model")

inputs = {'input': tf.saved_model.utils.build_tensor_info(model.input) }
outputs = { 'earnings': tf.saved_model.utils.build_tensor_info(model.output) }

signature_def = tf.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    K.get_session(),
    tags=[tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()

AttributeError: module 'tensorflow._api.v2.saved_model' has no attribute 'builder'